In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import random
import json
from scholarly import scholarly
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [ ]:
driver = webdriver.Chrome(r"C:\Users\COM01\Downloads\chromedriver_win32 (1)\chromedriver.exe", chrome_options=chrome_options)
driver.get('https://scholar.google.com/')
# log-in by self

In [ ]:
# read csv file 
# list scopus id into list_id
data_all = pd.read_csv(r"C:\Users\COM01\Downloads\5065-6330.csv",encoding='latin1')
list_id = ["UIY-locAAAAJ"]

In [ ]:
scholar = []
for id in list_id:
    driver.get(f'https://www.scopus.com/authid/detail.uri?authorId={id}')
    driver.execute_script("document.body.style.zoom='30%'")
    time.sleep(3)
    
# show 200 result
    
    try:
        driver.find_element(By.XPATH, '//*[@id="documents-panel"]/div[2]/div/div[2]/div/els-results-layout/els-paginator/nav/els-select/div/label/select/option[5]').click()
        time.sleep(3)
    except:
        pass

    button = driver.find_element(By.XPATH, '//*[@id="documents-panel"]/div[2]/div[2]/div[2]/div/els-results-layout/div[1]/ul/li[1]/div/div[1]/els-collapsible-panel-v2/div/button')
    button.click

    li_elements = driver.find_elements(By.XPATH, '//*[@id="documents-panel"]/div[2]/div/div[2]/div/els-results-layout/div[1]/ul/li')
    wait = WebDriverWait(driver, 10)

    for li in li_elements:
        try :
            button1 = li.find_element(By.XPATH, './/div/div/els-collapsible-panel-v2/div/button')
            driver.execute_script("arguments[0].click();", button1)
        except NoSuchElementException:
            pass
            
        time.sleep(3)


        

    author_name = driver.find_element(By.XPATH, '//h1[@data-testid="author-profile-name"]').text
    author_id = id 
    num_doc = driver.find_element(By.XPATH, '(//*/span[@data-testid="unclickable-count"])[2]').text
   
    
 
    research_list = []
    researchs = driver.find_elements(By.XPATH, "//*/els-results-layout//h4")
    for i in range(len(researchs)):
        research_list.append(researchs[i].text)
    
    date_list = []
    research_date = driver.find_elements(By.XPATH, '//*/els-results-layout/div[1]//span')
    for i in range(len(research_date)):
        date_list.append(research_date[i].text)

    abs_list = []
    abs = driver.find_elements(By.XPATH, '//*/els-results-layout//els-collapsible-panel-v2/section')
    for i in range(len(abs)):
        abs_list.append(abs[i].text)
    
    print(author_name, "Number of Documents :", num_doc)
    

    scopus = dict()
    scopus['author_id']  = author_id
    scopus['author_name'] = author_name
    scopus['documents_number'] = num_doc
    scopus['documents'] = list()
    for i in range(int(num_doc.split(" ")[0])):
        temp_dict = dict()
        temp_dict['title'] = research_list[i]
        temp_dict['date'] = date_list[i]
        temp_dict['abstraction'] = abs_list[i]
        scopus['documents'].append(temp_dict)

    scholar.append(scopus)

In [ ]:

scholar

In [ ]:
df1 = pd.DataFrame(scholar, columns = ['author_id', 'author_name', 'documents_number',
                                           'citiations' , 'h-index' , 'co-authors' , 'documents'])

In [ ]:
df1

In [ ]:
df1['documents'][0]

In [ ]:
type(df1['documents'][0])

In [ ]:
df1['documents'][0][1]

In [ ]:
df1=df1.explode('documents')
df1

In [ ]:
df1['main_title']=df1['documents'].apply(lambda v:v)
df1

In [ ]:
df1['main_abstract']=df1['main_title'].apply(lambda v:v)
df1

In [ ]:
df1['main_title'].apply(lambda v: v['title'])

In [ ]:
df1['Title']=df1['main_title'].apply(lambda v:v['title'])
df1

In [ ]:
df1['main_abstract'].apply(lambda v: v['abstraction'])

In [ ]:
df1['Abstraction']=df1['main_abstract'].apply(lambda v:v['abstraction'])
df1

In [ ]:
df1.drop(['documents' , 'main_title' , 'main_abstract'], axis=1)

In [ ]:
df1.to_xlsx('resercher_all.xlsx',index=False)